# ಮಾದರಿ 01: OpenAI SDK ಮೂಲಕ ತ್ವರಿತ ಚಾಟ್

ಈ ನೋಟ್ಬುಕ್ Microsoft Foundry Local ಜೊತೆಗೆ OpenAI SDK ಅನ್ನು ತ್ವರಿತ ಚಾಟ್ ಸಂವಹನಗಳಿಗೆ ಹೇಗೆ ಬಳಸುವುದು ಎಂಬುದನ್ನು ತೋರಿಸುತ್ತದೆ.

## ಅವಲೋಕನ

ಈ ಉದಾಹರಣೆ ತೋರಿಸುತ್ತದೆ:
- Foundry Local ಜೊತೆಗೆ OpenAI Python SDK ಬಳಕೆ
- Azure OpenAI ಮತ್ತು ಸ್ಥಳೀಯ Foundry ಸಂರಚನೆಗಳನ್ನು ಎರಡನ್ನೂ ನಿರ್ವಹಿಸುವುದು
- ಸರಿಯಾದ ದೋಷ ನಿರ್ವಹಣೆ ಮತ್ತು ಬ್ಯಾಕ್ಅಪ್ ತಂತ್ರಗಳನ್ನು ಅನುಷ್ಠಾನಗೊಳಿಸುವುದು
- ಸೇವಾ ನಿರ್ವಹಣೆಗೆ FoundryLocalManager ಬಳಕೆ


## ಪೂರ್ವಾಪೇಕ್ಷೆಗಳು

ನೀವು ಖಚಿತಪಡಿಸಿಕೊಳ್ಳಿ:
1. ಫೌಂಡ್ರಿ ಲೋಕಲ್ ಸ್ಥಾಪಿತ ಮತ್ತು ಚಾಲನೆಯಲ್ಲಿದೆ
2. ಒಂದು ಮಾದರಿ ಲೋಡ್ ಮಾಡಲಾಗಿದೆ (ಉದಾ., `foundry model run phi-4-mini`)
3. ಅಗತ್ಯ Python ಪ್ಯಾಕೇಜುಗಳು ಸ್ಥಾಪಿತವಾಗಿವೆ

### ಅವಲಂಬನೆಗಳನ್ನು ಸ್ಥಾಪಿಸಿ


In [ ]:
# Install required packages if not already installed
!pip install openai foundry-local-sdk

## ಅಗತ್ಯ ಗ್ರಂಥಾಲಯಗಳನ್ನು ಆಮದುಮಾಡಿ


In [ ]:
import os
import sys
from openai import OpenAI

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, will use manual configuration")

## ಸಂರಚನೆ

ನಿಮ್ಮ ಸಂರಚನೆಯನ್ನು ಹೊಂದಿಸಿ. ನೀವು ಕೆಳಗಿನವುಗಳಲ್ಲಿ ಯಾವುದಾದರೂ ಬಳಸಬಹುದು:
1. **ಆಜೂರ್ ಓಪನ್‌ಎಐ** (ಮೇಘಾಧಾರಿತ)
2. **ಫೌಂಡ್ರಿ ಲೋಕಲ್** (ಸ್ಥಳೀಯ ನಿರ್ಣಯ)

### ಆಯ್ಕೆ 1: ಆಜೂರ್ ಓಪನ್‌ಎಐ ಸಂರಚನೆ


In [ ]:
# Azure OpenAI Configuration (uncomment and set your values)
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"

### ಆಯ್ಕೆ 2: ಫೌಂಡ್ರಿ ಸ್ಥಳೀಯ ಸಂರಚನೆ


In [ ]:
# Foundry Local Configuration
MODEL_ALIAS = "phi-4-mini"  # Change this to your preferred model
BASE_URL = "http://localhost:8000"  # Default Foundry Local URL
API_KEY = ""  # Usually not needed for local

## ಕ್ಲೈಂಟ್ ಪ್ರಾರಂಭ

ನಿಮ್ಮ ಸಂರಚನೆಯ ಆಧಾರದ ಮೇಲೆ OpenAI ಕ್ಲೈಂಟ್ ಅನ್ನು ರಚಿಸಿ:


In [ ]:
def create_client():
    """Create and configure OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if azure_endpoint and azure_api_key:
        # Azure OpenAI path
        model = os.environ.get("MODEL", "your-deployment-name")
        client = OpenAI(
            base_url=f"{azure_endpoint}/openai",
            api_key=azure_api_key,
            default_query={"api-version": azure_api_version},
        )
        print(f"🌐 Using Azure OpenAI with model: {model}")
        return client, model
    
    # Foundry Local path
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            manager = FoundryLocalManager(MODEL_ALIAS)
            model_info = manager.get_model_info(MODEL_ALIAS)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            model = model_info.id
            print(f"🏠 Using Foundry Local SDK with model: {model}")
            return client, model
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{BASE_URL}/v1",
        api_key=API_KEY
    )
    model = MODEL_ALIAS
    print(f"🔧 Using manual configuration with model: {model}")
    return client, model

# Initialize the client
client, model = create_client()

## ಮೂಲ ಚಾಟ್ ಉದಾಹರಣೆ

ಸರಳ ಚಾಟ್ ಸಂವಹನವನ್ನು ಪರೀಕ್ಷಿಸೋಣ:


In [ ]:
def chat_with_model(prompt, max_tokens=128):
    """Send a chat message to the model and get a response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test with a simple prompt
prompt = "Say hello from Foundry Local and explain what you are in one sentence."
print(f"👤 User: {prompt}")
print("\n🤖 Assistant:")
response = chat_with_model(prompt)
print(response)

## ಇಂಟರಾಕ್ಟಿವ್ ಚಾಟ್ ಸೆಷನ್

ಮಾದರಿ ಹೇಗೆ ಪ್ರತಿಕ್ರಿಯಿಸುತ್ತದೆ ಎಂದು ನೋಡಲು ವಿಭಿನ್ನ ರೀತಿಯ ಪ್ರಾಂಪ್ಟ್‌ಗಳನ್ನು ಪ್ರಯತ್ನಿಸಿ:


In [ ]:
# Example prompts to try
example_prompts = [
    "What are the benefits of edge AI?",
    "Explain the difference between local and cloud AI inference.",
    "Write a short Python function to calculate the factorial of a number.",
    "What is Microsoft Foundry Local?"
]

for i, prompt in enumerate(example_prompts, 1):
    print(f"\n{'='*50}")
    print(f"Example {i}: {prompt}")
    print(f"{'='*50}")
    
    response = chat_with_model(prompt, max_tokens=200)
    print(response)

## ಉನ್ನತ ಬಳಕೆ: ಸ್ಟ್ರೀಮಿಂಗ್ ಪ್ರತಿಕ್ರಿಯೆಗಳು

ದೀರ್ಘ ಪ್ರತಿಕ್ರಿಯೆಗಳಿಗೆ, ಸ್ಟ್ರೀಮಿಂಗ್ ಉತ್ತಮ ಬಳಕೆದಾರ ಅನುಭವವನ್ನು ಒದಗಿಸಬಹುದು:


In [ ]:
def chat_with_streaming(prompt, max_tokens=300):
    """Send a chat message with streaming response."""
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        print("🤖 Assistant (streaming):")
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        return f"Error: {e}"

# Test streaming with a longer prompt
streaming_prompt = "Explain the advantages of running AI models locally on device versus in the cloud. Include privacy, latency, and cost considerations."
print(f"👤 User: {streaming_prompt}\n")
streaming_response = chat_with_streaming(streaming_prompt)

## ದೋಷ ನಿರ್ವಹಣೆ ಮತ್ತು ರೋಗನಿರ್ಣಯ

ಸೇವೆಯ ಸ್ಥಿತಿ ಮತ್ತು ಲಭ್ಯವಿರುವ ಮಾದರಿಗಳನ್ನು ಪರಿಶೀಲಿಸೋಣ:


In [ ]:
def check_service_health():
    """Check Foundry Local service health."""
    try:
        # Try to list available models
        models_response = client.models.list()
        models = [model.id for model in models_response.data]
        
        print("✅ Service is healthy")
        print(f"📋 Available models: {models}")
        print(f"🎯 Current model: {model}")
        
        if model in models:
            print("✅ Current model is available")
        else:
            print("⚠️ Current model may not be loaded")
            
    except Exception as e:
        print(f"❌ Service check failed: {e}")
        print("\n🔧 Troubleshooting tips:")
        print("1. Make sure Foundry Local is running")
        print("2. Check if a model is loaded: foundry service ps")
        print("3. Verify the endpoint URL is correct")

check_service_health()

## ಕಸ್ಟಮ್ ಪ್ರಾಂಪ್ಟ್ ಪರೀಕ್ಷೆ

ನಿಮ್ಮ ಸ್ವಂತ ಪ್ರಾಂಪ್ಟ್‌ಗಳನ್ನು ಪರೀಕ್ಷಿಸಲು ಕೆಳಗಿನ ಸೆಲ್ ಅನ್ನು ಬಳಸಿ:


In [ ]:
# Enter your custom prompt here
custom_prompt = "Write a haiku about artificial intelligence running on edge devices."

print(f"👤 User: {custom_prompt}\n")
custom_response = chat_with_model(custom_prompt, max_tokens=100)
print(f"🤖 Assistant: {custom_response}")

## ಸಾರಾಂಶ

ಈ ನೋಟ್ಬುಕ್ ತೋರಿಸಿದೆ:

1. **✅ ಕ್ಲೈಂಟ್ ಸೆಟ್‌ಅಪ್**: Foundry Local ಜೊತೆಗೆ OpenAI SDK ಅನ್ನು ಹೇಗೆ ಸಂರಚಿಸುವುದು
2. **✅ ಮೂಲಭೂತ ಚಾಟ್**: ಸರಳ ವಿನಂತಿ-ಪ್ರತಿಕ್ರಿಯೆ ಸಂವಹನಗಳು
3. **✅ ಸ್ಟ್ರೀಮಿಂಗ್**: ನೈಜ-ಸಮಯ ಪ್ರತಿಕ್ರಿಯೆ ಉತ್ಪಾದನೆ
4. **✅ ದೋಷ ನಿರ್ವಹಣೆ**: ಬಲವಾದ ದೋಷ ನಿರ್ವಹಣೆ ಮತ್ತು ರೋಗನಿರ್ಣಯ
5. **✅ ಸೇವಾ ಆರೋಗ್ಯ**: ಮಾದರಿ ಲಭ್ಯತೆ ಮತ್ತು ಸೇವಾ ಸ್ಥಿತಿಯನ್ನು ಪರಿಶೀಲಿಸುವುದು

### ಮುಂದಿನ ಹಂತಗಳು

- **ಉದಾಹರಣೆ 02**: Azure OpenAI ಬೆಂಬಲದೊಂದಿಗೆ ಸುಧಾರಿತ SDK ಏಕೀಕರಣ
- **ಉದಾಹರಣೆ 04**: Chainlit ಚಾಟ್ ಅಪ್ಲಿಕೇಶನ್ ನಿರ್ಮಾಣ
- **ಉದಾಹರಣೆ 05**: ಬಹು-ಏಜೆಂಟ್ ಸಂಯೋಜನಾ ವ್ಯವಸ್ಥೆಗಳು
- **ಉದಾಹರಣೆ 06**: ಬುದ್ಧಿವಂತ ಮಾದರಿ ಮಾರ್ಗದರ್ಶನ

### Foundry Local ನ ಪ್ರಮುಖ ಲಾಭಗಳು

- 🔒 **ಗೌಪ್ಯತೆ**: ಡೇಟಾ ನಿಮ್ಮ ಸಾಧನವನ್ನು ಎಂದಿಗೂ ಬಿಟ್ಟು ಹೋಗುವುದಿಲ್ಲ
- ⚡ **ವೇಗ**: ಕಡಿಮೆ ವಿಳಂಬದ ಸ್ಥಳೀಯ ನಿರ್ಣಯ
- 💰 **ಖರ್ಚು**: ಯಾವುದೇ API ಬಳಕೆ ವೆಚ್ಚಗಳಿಲ್ಲ
- 🔌 **ಆಫ್‌ಲೈನ್**: ಇಂಟರ್ನೆಟ್ ಸಂಪರ್ಕವಿಲ್ಲದೆ ಕಾರ್ಯನಿರ್ವಹಿಸುತ್ತದೆ
- 🛠️ **ಸಮ್ಮಿಲನಶೀಲತೆ**: OpenAI-ಸಮ್ಮಿಲಿತ API


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸ್ವೀಕರಣ**:  
ಈ ದಸ್ತಾವೇಜು AI ಅನುವಾದ ಸೇವೆ [Co-op Translator](https://github.com/Azure/co-op-translator) ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ನಿಖರತೆಯಿಗಾಗಿ ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ದೋಷಗಳು ಅಥವಾ ಅಸತ್ಯತೆಗಳು ಇರಬಹುದು ಎಂದು ದಯವಿಟ್ಟು ಗಮನಿಸಿ. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಸ್ತಾವೇಜನ್ನು ಅಧಿಕೃತ ಮೂಲವೆಂದು ಪರಿಗಣಿಸಬೇಕು. ಮಹತ್ವದ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪು ಅರ್ಥಮಾಡಿಕೊಳ್ಳುವಿಕೆ ಅಥವಾ ತಪ್ಪು ವಿವರಣೆಗಳಿಗೆ ನಾವು ಹೊಣೆಗಾರರಾಗುವುದಿಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
